In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()
import sklearn as sl
import nltk
import pickle

from google.colab import drive
drive.mount('/content/drive/')

filename = '/content/drive/My Drive/기계학습/train.csv'
data = pd.read_csv(filename)

texts = []
label=[]

for labels in data['label']:
    label.append(labels)
for message in data['message']:
    texts.append(message)
# x_raw.append(' '.join(i for i in line.split()[1:]))


print(len(label))
print(len(texts))

from keras.layers import SimpleRNN, Embedding, Dense, LSTM
from keras.models import Sequential

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# number of words used as features
max_features = 10000
# cut off the words after seeing 500 words in each document(email) 
maxlen = 500



tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print("Found {0} unique words: ".format(len(word_index)))
print(sequences)
print(word_index)
X = pad_sequences(sequences, maxlen=maxlen)

print("data shape: ", X.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,label, test_size = 0.1, random_state = 42)
x_test = X_test.reshape(400,500,1)
x_train = X_train.reshape(3600,500,1)
print(X_train)
print(len(X_train))
print(len(X_train[0]))
print(y_train)
print(X_train.dtype)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_rnn = model.fit(X_train, y_train, epochs=10, batch_size=60,validation_split=0.2)

acc = history_rnn.history['acc']
val_acc = history_rnn.history['val_acc']
loss = history_rnn.history['loss']
val_loss = history_rnn.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, '-', color='orange', label='training acc')
plt.plot(epochs, val_acc, '-', color='blue', label='validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

plt.plot(epochs, loss, '-', color='orange', label='training acc')
plt.plot(epochs, val_loss,  '-', color='blue', label='validation acc')
plt.title('Training and validation loss')
plt.legend()
plt.show()
model.summary()
pred = model.predict_classes(X_test)
acc = model.evaluate(X_test, y_test)
proba_rnn = model.predict_proba(X_test)

##################################################
# ㅈ

In [ ]:
filename = '/content/drive/My Drive/기계학습/leaderboard_test_file.csv'
result_path='/content/drive/My Drive/기계학습/result.csv'
data = pd.read_csv(filename)


texts=[]

from pandas import DataFrame
for message in data['message']:
    texts.append(message)

print(texts)

# number of words used as features
max_features = 10000
# cut off the words after seeing 500 words in each document(email) 
maxlen = 500



tokenizer = Tokenizer()
tokenizer.word_index = word_index
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

print(sequences)
# word_index = tokenizer.word_index
print("Found {0} unique words: ".format(len(word_index)))

eval = pad_sequences(sequences, maxlen=maxlen)

print(eval.shape)
eval.reshape(300,500,1)
model.summary()
score = model.predict_classes(eval)
print(score)
score = pd.DataFrame(score)
score.to_csv(result_path,index = False)